## Объединение файлов на отдельных листах Excel

Задача данного примера - агрегировать информацию из набора файлов excel в один файл, помещая данные из каждого следующего файла на отдельную вкладку. Для примера используем выгрузки с курсом валют за некоторый период по нескольким валютам. Исходные файлы находятся в папке "Examples/Exchange rates".

Для начала импортируем необходимые нам библиотеки, в том числе:

* xlwings - для работы с excel
* os - для работы с файловой системой (с папками в windows)

In [1]:
import xlwings as xw
import os

### Библиотека OS

Библиотека OS предоставляет набор функций для работы с операционной системой, в том числе функции для работы с файловой системой, например, для получения списка файлов в папке, для создания, переименования и удаления папок.

Функции, которые нам понадобятся в этом примере:

**1) ``os.listdir('folder path')``**<br>
Функция для получения списка файлов и папок в указанной папке.
Возвращает список имен файлов/папок вида **['file1.xlsx', 'file2.xlsx', 'Folder1']**

In [2]:
os.listdir('../Examples') # Знак ".." используется для возврата в каталог верхнерго уровня

['Exchange rates', 'GDP.xlsx', 'Sales by region.xlsx']

**2) ``os.path.splitext('file path')``**<br>
Функция разбивает полный путь к файлу (например '/folder/file_name.xlsx') на две части:

1. путь + имя файла ('/folder/file_name')
2. расширение файла ('xlsx')

Возвращает кортеж из пути и расширения вида ***('folder/file_name', 'xlsx')***.

In [3]:
os.path.splitext('../Examples/Exchange rates/USD.xlsx')

('../Examples/Exchange rates/USD', '.xlsx')

Рассмотрим пример чтения и переноса данных из файла excel в новую книгу на примере выгрузки курса доллара.<br>
Создадим новую книгу excel при помощи библиотеки xlwings и поместим ссылку на нее в переменную new_book.

In [4]:
new_book = xw.Book()

Далее откроем файл с выгрузкой и поместим ссылку на него в переменную source_book.

In [5]:
source_book = xw.books.open('../Examples/Exchange rates/USD.xlsx')

Скопируем лист с выгрузкой курсов долл. в новую книгу при помощи функции ``copy()``, указав имя нового листа в параметре ``name`` и его позицию в новой книге через параметр ``after``.

In [6]:
source_book.sheets[0].copy(name='USD', after=new_book.sheets[0])

<Sheet [Book1]USD>

Лист скопирован в новый файл. На практике такая автоматизация как правило имеет смысл для большого количества файлов. Поэтому в конечном итоге мы бы хотели получить цикл, который перебирает все файлы excel в папке и копирует из них листы с курсами в новый файл.

Для начала получим список всех файлов в папке с выгрузками.

In [7]:
files = os.listdir('../Examples/Exchange rates')
print(files)

['CHF.xlsx', 'EUR.xlsx', 'JPY.xlsx', 'not excel file.txt', 'USD.xlsx']


Как видим в полученном списке не только искомые файлы с выгрузками, но также лишний текстовый файл 'not excel file.txt', а также временный файл '~$USD.xlsx', который создался, когда мы открыли файл USD.xlsx. Чтобы отфильтровать их воспользуемся условием и проверим, что:
1. Расширение файла относится к стандартным файлам excel (xls, xlsx)
2. Название файла не начинается на знак ~

Для начала разделим полное название каждого файла на собственно имя файла и расширение и присвоим их отдельным переменным file_name и file_ext.

Чтобы проверить является ли расширение одним из тех, которые используются excel можно воспользоваться оператором ``in``, который позволяет проверить входит ли определенное значение в список.

Например

In [8]:
lst = ['a', 'b', 'd']

In [9]:
'b' in lst # 'b' входит в список

True

In [10]:
'c' in lst # а 'c' не входит

False

Для проверки того, что название файла начинается с тильды воспользуемся свойством того, что все строковые объекты в python итерируемые, и просто сравним первый элемент (с индексом 0) в имени каждого файла со знаком "~".

In [11]:
for file in files: # перебираем все файлы в папке
    file_name = os.path.splitext(file)[0] # первый элемент в кортеже, возвращаемом функцией splitext, - имя файла
    file_ext = os.path.splitext(file)[1] # второй элемент - расширение файла
    if (file_ext in ['.xls', '.xlsx']) and (file_name[0] != '~'):
        print(file_name)

CHF
EUR
JPY
USD


Как видим, мы получили только имена файлов с выгрузками. Теперь сохраним адрес папки с выгрузками в отдельную переменную source (она еще пригодится нам позже для оформления нашего скрипта в функцию) и получим полный адрес каждого файла (full_path), объединив путь к папке с выгрузками с названиями каждого excel файла, перебираемого в цикле.

Далее повторим ранее использованную логику для копирования листов в excel в цикле. Будем открывать каждый файл, который мы определили как файл excel, копировать из него первый (и единственный) лист и вставлять его в новую книгу.

Для корректной работы примера, закройте файлы Book1.xlsx и USD.xlsx (и все другие файлы из папки с выгрузками, если они открыты).

In [12]:
source = '../Examples/Exchange rates/'
new_book = xw.Book()

for file in files:
    file_name, file_ext = os.path.splitext(file) # можно присвоить элементы кортежа сразу двум переменным одной инструкцией
    if (file_ext in ['.xls', '.xlsx']) and (file_name[0] != '~'):
        full_path = source + file
        source_book = xw.books.open(full_path)
        source_book.sheets[0].copy(name=file_name, after=new_book.sheets[0])
        source_book.close() # не забываем закрывать не используемые файлы!

Все вкладки скопированы в новую книгу. Для удобства использования этого скрипта, оформим его в виде функции. Пусть наша функция принимает адрес папки с выгрузками, которые необходимо объединить, а также адрес и имя файла, в который нужно сохранить результат. Причем второй параметр можно сделать опциональным: если он не указан, файл будет сохраняться под каким-нибудь стандартным именем (например, "combined.xlsx") в папке с нашим скриптом (в данном случае с jupyter noteboook).

In [13]:
def combine_excels(source, dest='combined.xlsx'):
    
    import xlwings as xw
    import os
    
    files = os.listdir(source)
    new_book = xw.Book()
    
    for file in files:
        file_name, file_ext = os.path.splitext(file)
        if (file_ext in ['.xls', '.xlsx']) and (file_name[0] != '~'):
            full_path = source + file
            source_book = xw.books.open(full_path)
            source_book.sheets[0].copy(name=file_name, after=new_book.sheets[0])
            source_book.close()
            
    new_book.save(dest)

Закроем все файлы excel и протестируем нашу функцию

In [15]:
combine_excels('../Examples/Exchange rates/')

В качестве последнего штриха можно скрыть процесс открывания файлов, копирования данных и перевести все операции в excel в фоновый режим. Для этого перед выполнением цикла нужно найти объект ``Application`` (приложение excel), с которым мы работаем, и установить свойство ``visible`` для этого объекта равным **False**, а в после выполнения скрипта, вернуть значение по умолчанию (**True**).

In [16]:
def combine_excels(source, dest='combined.xlsx'):
    
    import xlwings as xw
    import os
    
    files = os.listdir(source)
    new_book = xw.Book()
    
    app = xw.apps.active # найти активное приложение Excel
    app.visible = False # отключить для него обновление экрана
    
    for file in files:
        file_name, file_ext = os.path.splitext(file)
        if (file_ext in ['.xls', '.xlsx']) and (file_name[0] != '~'):
            full_path = source + file
            source_book = xw.books.open(full_path)
            source_book.sheets[0].copy(name=file_name, after=new_book.sheets[0])
            source_book.close()
            
    new_book.save(dest)
    app.visible = True # включить обратно обновление экрана для Excel

In [17]:
combine_excels('../Examples/Exchange rates/')

### Задание

Для того, чтобы немного попрактиковать полученные знания, попробуйте написать схожую функцию, которая также будет собирать данные из каждого excel файла в указанной папке, но в отличие от рассмотренного примера, данные будут не вставляться отдельными листами в новую книгу, а собираться в единый pandas DataFrame. А затем объединенный массив с данными по всем валютам должен сохраняться в отдельный файл excel.